In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
weather = pd.read_csv('./fullweather.csv')

In [3]:
weather

,Local time in Jeju (airport),T,WW
0,2019-04-16 1:00,11.6,NaN
1,2019-04-16 2:00,11.4,NaN
2,2019-04-16 3:00,11.4,NaN
3,2019-04-16 4:00,11.1,NaN
4,2019-04-16 5:00,11.2,NaN
...,...,...,...
74759,2021-06-02 20:00,22.0,NaN
74760,2021-06-02 21:00,22.0,NaN
74761,2021-06-02 22:00,22.1,NaN
74762,2021-06-02 23:00,22.6,NaN


In [4]:
weather['Local time in Jeju (airport)'] = pd.to_datetime(weather['Local time in Jeju (airport)'], format='%Y-%m-%d %H:%M', errors='raise')
weather['Date'] = weather['Local time in Jeju (airport)'].dt.date
weather['WW'][weather['WW'].notnull()] = 1
weather['WW'][weather['WW'].isnull()] = 0
weather['WW'] = pd.to_numeric(weather['WW'])


C:\Users\mllab\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mllab\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
temp_rain = weather.groupby('Date',as_index=False).mean()
temp_rain.rename(columns={'WW':'rain'},inplace=True)
temp_rain['Date'] = temp_rain['Date'].astype(str)

In [6]:
temp_rain

,Date,T,rain
0,2019-04-16,15.939130,0.021739
1,2019-04-17,15.412500,0.083333
2,2019-04-18,15.431250,0.000000
3,2019-04-19,16.289583,0.000000
4,2019-04-20,16.446875,0.000000
...,...,...,...
775,2021-05-30,20.061458,0.000000
776,2021-05-31,20.938542,0.000000
777,2021-06-01,21.856250,0.000000
778,2021-06-02,22.740625,0.000000


In [7]:
weather

,Local time in Jeju (airport),T,WW,Date
0,2019-04-16 01:00:00,11.6,0.0,2019-04-16
1,2019-04-16 02:00:00,11.4,0.0,2019-04-16
2,2019-04-16 03:00:00,11.4,0.0,2019-04-16
3,2019-04-16 04:00:00,11.1,0.0,2019-04-16
4,2019-04-16 05:00:00,11.2,0.0,2019-04-16
...,...,...,...,...
74759,2021-06-02 20:00:00,22.0,0.0,2021-06-02
74760,2021-06-02 21:00:00,22.0,0.0,2021-06-02
74761,2021-06-02 22:00:00,22.1,0.0,2021-06-02
74762,2021-06-02 23:00:00,22.6,0.0,2021-06-02


In [8]:
data = pd.read_csv('fulldata.csv', encoding='euc-kr')
data.drop(['year','month'],axis=1,inplace=True)

In [9]:
place = pd.read_csv('station.csv', encoding='euc-kr') #길거리에 Iot 있는 킥보드가 있는 장소

In [10]:
data['Xpos'] = None
data['Ypos']= None

#위치 추가
for i in range(len(data)):
    for j in range(len(place)):
        if(data['rent_station'].iloc[i] == place['id'].iloc[j]):
            data['Xpos'].iloc[i] = float(round(place['x_pos'].iloc[j],3))
            data['Ypos'].iloc[i] = float(round(place['y_pos'].iloc[j],3))

C:\Users\mllab\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
data['Xpos'] = data['Xpos'].astype('float')
data['Ypos'] = data['Ypos'].astype('float')

In [12]:
#data = data.iloc[138:] #2019-08-01~
#data = data.iloc[385:] #2019-12-01~
#data = data.iloc[592:] #2020-04-01~

In [13]:
#data = data.iloc[887:] #일정한 수요가 생기는 2020-04-30일 이후 데이터를 자름

## 지역 여러개 데이터 분할

In [14]:
NineSector_data = data.copy()
TwoSector_data = data.copy()
OneSector_data = data.copy()
FourSector_data = data.copy()
OneSector_data['Sector'] = 0

# ========================================================

In [15]:
def preprocessing(data):
    data['rent_date'] = pd.to_datetime(data['rent_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    data['year'] =data['rent_date'].dt.year
    data['month'] =data['rent_date'].dt.month
    data['day'] =data['rent_date'].dt.day
    data['day_name'] = data['rent_date'].dt.day_name()
    data = data.sort_values(by='rent_date')
    data['rent_date'] = data['rent_date'].astype(str)
    data['T'] = None
    data['rain'] = None
    return data

In [16]:
def temp_rain_preprocessing(data,length,temp_length):
    for i in range(temp_length):
        for j in range(length):
            if(data['rent_date'].iloc[j] == temp_rain['Date'].iloc[i]):
                data['T'].iloc[j] = temp_rain['T'].iloc[i]
                data['rain'].iloc[j] = temp_rain['rain'].iloc[i]
                if(data['rain'].iloc[j]>0):
                    data['rain'].iloc[j] = 0
                else:
                    data['rain'].iloc[j] = 1
    data['T'] = data['T'].astype(int)
    data['rent_date']=pd.to_datetime(data['rent_date'])
    return data

In [17]:
def weekend_weekday_preprocessing(data,length):
    data['weekend'] = None
    for i in range(length):
        if(('Friday' == data['day_name'].iloc[i]) or ('Saturday'== data['day_name'].iloc[i]) or ('Sunday' == data['day_name'].iloc[i])):
            data['weekend'].iloc[i] = 1
        else:
            data['weekend'].iloc[i] = 0

    data['weekend']=data['weekend'].astype(int)
    return data

In [18]:
def dummy_preprocessing(data):
    data['weekday'] = data['rent_date'].dt.weekday
    data = pd.get_dummies(data,columns=['weekday'])
    data = pd.get_dummies(data,columns=['month'])
    data = pd.get_dummies(data,columns=['year'])
    return data

In [19]:
def WholePreprocessing(data):
    data = data.groupby(['rent_date','Sector'],as_index=False).sum()
    length = len(data)
    temp_length = len(temp_rain)
    data = preprocessing(data)
    data = temp_rain_preprocessing(data,length,temp_length)
    data = weekend_weekday_preprocessing(data,length)
    data = dummy_preprocessing(data)
    data.drop(['day_name','Xpos','Ypos'],axis=1,inplace=True)
    return data

# 지역 한개 모델

In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import make_scorer, mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from math import sqrt
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from mlens.ensemble import BlendEnsemble
from sklearn.linear_model import LinearRegression
from mlens.ensemble import SuperLearner
from sklearn.metrics import mean_absolute_error

[MLENS] backend: threading


In [21]:
from scipy.signal import savgol_filter
def Smoothing(data):
    x = data['rent_number'].to_numpy()
    x = savgol_filter(x, 13, 2)
    data['rent_number'] = x
    data['rent_number'] = data['rent_number'].astype(float)
    return data

In [22]:
OneSector_data = OneSector_data.groupby(['rent_date','Sector'],as_index=False).sum()
OneSector_data = WholePreprocessing(OneSector_data)
OneSector_data = Smoothing(OneSector_data)

C:\Users\mllab\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
OneSector_data['rain'] = OneSector_data['rain'].astype(str).astype(int)

In [24]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
Two_points = pd.DataFrame().reset_index(drop=True)
Two_points['Xpos'] = TwoSector_data['Xpos']
Two_points['Ypos'] = TwoSector_data['Ypos']
kmeans = KMeans(n_clusters=2)
kmeans.fit(Two_points)
TwoSector_data['Sector'] = kmeans.labels_

In [25]:
TwoSector_data = WholePreprocessing(TwoSector_data)
TwoSector_data = Smoothing(TwoSector_data)

C:\Users\mllab\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
TwoSector_data['rain'] = TwoSector_data['rain'].astype(str).astype(int)

In [27]:
from sklearn.cluster import KMeans
Four_points = pd.DataFrame().reset_index(drop=True)
Four_points['Xpos'] = FourSector_data['Xpos']
Four_points['Ypos'] = FourSector_data['Ypos']
kmeans = KMeans(n_clusters=4)
kmeans.fit(Four_points)
FourSector_data['Sector'] = kmeans.labels_

In [28]:
FourSector_data = WholePreprocessing(FourSector_data)
FourSector_data = Smoothing(FourSector_data)

C:\Users\mllab\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
FourSector_data['rain'] = FourSector_data['rain'].astype(str).astype(int)

In [30]:
FourSector_data[-8:-4]

,rent_date,Sector,rent_number,day,T,rain,weekend,weekday_0,weekday_1,weekday_2,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2019,year_2020,year_2021
1795,2021-06-02,2,28.279720,2,22,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
1796,2021-06-02,3,28.370629,2,22,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
1793,2021-06-02,0,26.668332,2,22,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
1794,2021-06-02,1,24.924076,2,22,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1


In [32]:
Four_X_test

,Sector,day,T,rain,weekend,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2019,year_2020,year_2021
1679,0,4,18,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1680,1,4,18,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1685,2,5,17,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1686,3,5,17,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
for i in range(2, 31):
    print(i, '번째')
    X_data = OneSector_data.drop(['rent_date','rent_number'],axis=1)
    y_data = OneSector_data['rent_number']
    X_train = X_data.iloc[:-i]
    y_train = y_data.iloc[:-i]
    X_test = X_data.iloc[-i:-i+1]
    y_test = y_data.iloc[-i:-i+1]
    xlf = lgb.LGBMRegressor()
    xlf.fit(X_train, y_train)
    pred = xlf.predict(X_test)
    pred = pred.astype(float)
    print('분할 없음: ', mean_absolute_error(y_test, pred))
    
    Two_X_data = TwoSector_data.drop(['rent_date','rent_number'],axis=1)
    Two_y_data = TwoSector_data['rent_number']
    Two_X_train = Two_X_data.iloc[:-2*i]
    Two_y_train = Two_y_data.iloc[:-2*i]
    Two_X_test = Two_X_data.iloc[-2*i:(-2*i)+2]
    Two_y_test = Two_y_data.iloc[-2*i:(-2*i)+2]
    Two_xlf = lgb.LGBMRegressor()
    Two_xlf.fit(Two_X_train, Two_y_train)
    Two_pred = Two_xlf.predict(Two_X_test)
    Two_pred = Two_pred.astype(float)
    print('2분할: ', mean_absolute_error(Two_y_test, Two_pred))
    
    Four_X_data = FourSector_data.drop(['rent_date','rent_number'],axis=1)
    Four_y_data = FourSector_data['rent_number']
    Four_X_train = Four_X_data.iloc[:-4*i]
    Four_y_train = Four_y_data.iloc[:-4*i]
    Four_X_test = Four_X_data.iloc[-4*i:(-4*i)+4]
    Four_y_test = Four_y_data.iloc[-4*i:(-4*i)+4]
    Four_xlf = lgb.LGBMRegressor()
    Four_xlf.fit(Four_X_train, Four_y_train)
    Four_pred = Four_xlf.predict(Four_X_test)
    Four_pred = Four_pred.astype(float)
    print('4분할: ', mean_absolute_error(Four_y_test, Four_pred))
    
    print('-------------------------------------------------------')

2 번째
분할 없음:  7.725138174156356
2분할:  5.5505915803389065
4분할:  2.0501804925776606
-------------------------------------------------------
3 번째
분할 없음:  15.875635804081398
2분할:  11.047637679378
4분할:  5.553812508930471
-------------------------------------------------------
4 번째
분할 없음:  7.473519507130845
2분할:  9.2550627747735
4분할:  3.4008111345621774
-------------------------------------------------------
5 번째
분할 없음:  6.324367032816426
2분할:  9.367319417761557
4분할:  3.322199370087027
-------------------------------------------------------
6 번째
분할 없음:  3.7036772195950647
2분할:  17.119088143939415
4분할:  4.658889448070193
-------------------------------------------------------
7 번째
분할 없음:  1.5468577292365495
2분할:  5.801648383904194
4분할:  0.780654006308235
-------------------------------------------------------
8 번째
분할 없음:  4.588753409509167
2분할:  6.94317651111626
4분할:  4.849334329237231
-------------------------------------------------------
9 번째
분할 없음:  4.880502486189329
2분할:  12.3641783559206